In [1]:
import matplotlib.pyplot as plt
import numpy as np
import caiman as cm
import os
from skimage import io

# Importing datajoint tables
import sys
sys.path.append("..\\")

import login
login.connect()
from schema import common_img

# Importing custom_caiman functions
sys.path.append("..\\..\\Caiman\\custom scripts")
import standard_pipeline.place_cell_pipeline as pipe

Connecting hheise@130.60.53.48:3306


Using TensorFlow backend.


In [ ]:
# Memory-mapped files are created from MotionCorrection() entries
common_img.MotionCorrection() & 'username="hheise"'

In [ ]:
# Get a quick overview of the differences between session templates and choose a session for parameter tuning
# that is representative for the others

%matplotlib qt

key = dict(username='hheise', mouse_id=95)

templates, days = (common_img.MotionCorrection & key).fetch('template', 'day')
n_frames = (common_img.ScanInfo & key).fetch('nr_frames')
n_trials = [len(common_img.RawImagingFile & dict(**key, day=x)) for x in days]

n_cols = 4
n_rows = int(np.ceil(len(templates)/n_cols))

fig, ax = plt.subplots(n_rows, n_cols)

count = 0
for row in range(n_rows):
    for col in range(n_cols):
        if count < len(templates):
            ax[row, col].imshow(templates[count])
            ax[row, col].set_title(f'{days[count]}, {n_trials[count]} trials, {n_frames[count]} frames')
            count += 1

In [3]:
# Create memory-mapped file of the example session and load into memory
key = dict(username='hheise', mouse_id=95, day='2021-07-08')

params = (common_img.MotionCorrection & key).get_parameter_obj()

common_img.MemoryMappedFile.populate(key)
images, mmap_dir = (common_img.MemoryMappedFile & key).load_mmap_file()

Creating memory mapped file...


In [ ]:
(common_img.MotionParameter & key)

In [ ]:
common_img.MemoryMappedFile().delete()

In [2]:
# If this session has been segmented before and parameters should be changed, load the previous parameters here and
# enter them into the next code blocks
old_params = (common_img.CaimanParameter & 'mouse_id=95')
old_params

"username Unique username, should be the UZH shortname",mouse_id ID of mouse (unique per investigator),"caiman_id index for unique parameter set, base 0",p order of the autoregressive system (should always be 1),nb number of global background components,"merge_thr merging threshold, max correlation of components allowed before merged","rf half-size of the patches in pixels. e.g., if rf=25, patches are 50x50. -1 if no patches.",stride_cnmf amount of overlap between the patches in pixels,k number of components per patch,g_sig expected half size of neurons in pixels,method_init initialization method (if analyzing dendritic data using 'sparse_nmf'),ssub spatial subsampling during initialization,tsub temporal subsampling during initialization,snr_lowest rejection threshold of signal-to-noise ratio,snr_thr upper threshold of signal-to-noise ratio,rval_lowest rejection threshold of spatial correlation of ROI mask vs trace,rval_thr upper threshold of spatial correlation of ROI mask vs trace,cnn_lowest rejection threshold of CNN-based classifier,cnn_thr upper threshold for CNN based classifier,flag_auto flag for using provided or computed percentile as baseline fluorescence. If,quantile_min Quantile to use as baseline fluorescence. Only used if flag_auto is False.,frame_window Sliding window size of fluorescence normalization,weight_thresh Threshold of giving an ROI ownership over a pixel
hheise,95,0,1,3,0.75,25,10,20,5,greedy_roi,2,2,5.0,11.0,-1.0,0.78,0.1,0.9,1,8,1000,0.005


In [4]:
# Set and adjust parameters of initial source extraction

p = 1    # order of the autoregressive system
gnb = 3  # number of global background components (3)
merge_thr = 0.75  # merging threshold, max correlation allowed (0.86)
rf = 25  # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
stride_cnmf = 6  # amount of overlap between the patches in pixels (20)
K = 20  # number of components per patch (10)
gSig = [5, 5]  # expected half-size of neurons in pixels [X, Y] (has to be int, not float!)
ssub = 2  # spatial subsampling during initialization
tsub = 2  # temporal subsampling during intialization


opts_dict = {'nb': gnb,
             'rf': rf,
             'K': K,
             'gSig': gSig,
             'stride': stride_cnmf,
             'rolling_sum': True,
             'merge_thr': merge_thr,
             'only_init': True,
             'ssub': ssub,
             'tsub': tsub}

params = params.change_params(opts_dict)

In [5]:
%matplotlib qt

# # RUN SOURCE EXTRACTION

# Start cluster for parallel processing
c, dview, n_processes = cm.cluster.setup_cluster(backend='local', n_processes=None, single_thread=False)

# # # Run source extraction
cnm = pipe.run_source_extraction(images, params, dview=dview)
pipe.save_cnmf(cnm, path=os.path.join(mmap_dir, 'cnm_pre_selection.hdf5'), verbose=False, overwrite=True)

# Load local correlation image
try:
    cnm.estimates.Cn = io.imread(mmap_dir + r'\local_correlation_image.tif')
except FileNotFoundError:
    pipe.save_local_correlation(images, mmap_dir)
    cnm.estimates.Cn = io.imread(mmap_dir + r'\local_correlation_image.tif')
pipe.save_cnmf(cnm, path=os.path.join(mmap_dir, 'cnm_pre_selection.hdf5'), verbose=False, overwrite=True)

# Plot and save contours of all components
cnm.estimates.plot_contours(img=cnm.estimates.Cn, display_numbers=False)
plt.tight_layout()
fig = plt.gcf()
fig.set_size_inches((10, 10))
pipe.save_cnmf(cnm, path=os.path.join(mmap_dir, 'cnm_pre_selection.hdf5'), verbose=False, overwrite=True)

C:\Anaconda3\envs\caiman\lib\site-packages\sklearn\decomposition\_nmf.py:1091: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
C:\Anaconda3\envs\caiman\lib\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


Saved local correlation image at C:\Users\hheise\Datajoint\temp\20220324_161820\local_correlation_image.tif.


C:\Anaconda3\envs\caiman\lib\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [11]:
# # RUN EVALUATION

# evaluation parameters
min_SNR = 8.5  # signal to noise ratio for accepting a component (default 2)
SNR_lowest = 3.4
rval_thr = 0.78  # space correlation threshold for accepting a component (default 0.85)
rval_lowest = -1
cnn_thr = 0.9  # threshold for CNN based classifier (default 0.99)
cnn_lowest = 0.1  # neurons with cnn probability lower than this value are rejected (default 0.1)

cnm.params.set('quality', {'SNR_lowest': SNR_lowest,
                           'min_SNR': min_SNR,
                           'rval_thr': rval_thr,
                           'rval_lowest': rval_lowest,
                           'use_cnn': True,
                           'min_cnn_thr': cnn_thr,
                           'cnn_lowest': cnn_lowest})
cnm = pipe.run_evaluation(images, cnm, dview=dview)

cnm.estimates.plot_contours(img=cnm.estimates.Cn, idx=cnm.estimates.idx_components, display_numbers=True)

USING MODEL:C:\Users\hheise\caiman_data\model\cnn_model.json
1981/1981 [==============================] - 1s 411us/sample


In [ ]:
# Plot contours of accepted vs rejected components
cnm.estimates.plot_contours(img=cnm.estimates.Cn, idx=cnm.estimates.idx_components, display_numbers=True)

In [ ]:
# Plot accepted and rejected components in a detailed view (spatial, temporal and background components)
cnm.estimates.view_components(images, img=cnm.estimates.Cn, idx=cnm.estimates.idx_components)
cnm.estimates.view_components(images, img=cnm.estimates.Cn, idx=cnm.estimates.idx_components_bad)

In [13]:
# # MANUAL ADJUSTMENT

# Check single component (input index is from plot_contours() label - 1 (base 0 index))
pipe.check_eval_results(cnm, cnm.estimates.idx_components_bad[779], plot_contours=True)

Checking component 1027...

Component 1027 did not get accepted. 

	Checking thresholds:

	SNR of 5.63 exceeds lower threshold of 3.4, but not upper threshold of 8.5

	R-value of 0.25 exceeds lower threshold of -1, but not upper threshold of 0.78

	CNN-value of 0.37 exceeds lower threshold of 0.1, but not upper threshold of 0.9

Result: Component 1027 got rejected because it met all lower, but no upper thresholds!




In [ ]:
# Plot trace to check for real cells/transients (index is the global component index returned 
# from the check_eval_results() function)
plt.figure()
idx = 976
plt.plot(cnm.estimates.C[idx])

In [ ]:
# Manually accept cells if necessary (use indices of idx_components_bad) -> after execution, call plot_contours() again to 
# update cell indices
cnm = pipe.accept_cells(cnm, [109,111,143])

In [ ]:
# Manually reject cells if necessary (use indices of idx_components) -> after execution, call plot_contours() again to 
# update cell indices
cnm = pipe.reject_cells(cnm, [41, 42])

In [14]:
# ENTER PARAMETERS INTO DATABASE

# If parameters are tuned, enter them into the database
# Adjust additional parameters if needed

new_param_entry = dict(
    username='hheise',
    mouse_id=95,
    caiman_id=1,
    # Parameters from above. Check that the accepted params are in local memory right now (execute cells again just in case)
    p=p,
    nb=gnb,
    merge_thr=merge_thr,
    rf=rf,
    stride_cnmf=stride_cnmf,
    k=K,
    g_sig=gSig[0],
    ssub=ssub,
    tsub=tsub,
    snr_lowest=SNR_lowest,
    snr_thr=min_SNR,
    rval_lowest=rval_lowest,
    rval_thr=rval_thr,
    cnn_lowest=cnn_lowest,
    cnn_thr=cnn_thr,
    frame_window=1000)

common_img.CaimanParameter().insert1(new_param_entry)


In [ ]:
# Check contents of the parameter table
common_img.CaimanParameter()

In [ ]:
cnm = pipe.load_cnmf(mmap_dir, 'cnm_pre_selection.hdf5')

In [ ]:
common_img.MemoryMappedFile()

In [ ]:
(common_img.MemoryMappedFile & 'mouse_id=95').delete()

In [ ]:
c, dview, n_processes = cm.cluster.setup_cluster(backend='local', n_processes=None, single_thread=False)